In [1]:
import requests
import csv
from ltr.data import CorpusApi, Config
import numpy as np

In [4]:
queries = CorpusApi.getValidationQueriesAsDict()

In [16]:
for i, key in enumerate(list(queries.keys())[:]):
    try:
        # get query
        query = queries[key]
        
        # get result from solr
        request = {
            "fields": "id, score",
            "limit": 500,
            "params": {
                "qf": "title headings body",
                "defType": "dismax",
                "q": query
            }
        }
        
        response = requests.post(f'http://localhost:8983/solr/thesis-ltr/select', json=request)
        
        queryResult = [doc for doc in response.json()["response"]["docs"]]
        
        # write resultfile for trec eval
        with open('evaluation/bm25_eval.results', 'a') as file:
            for j, doc in enumerate(queryResult):
                line = " ".join([str(key), "Q0", str(doc['id']), str(j + 1), str(doc['score']), "STANDARD"])
                file.write(line)
                file.write('\n')
        
        print(f'Processed queries: {i + 1}/{len(list(queries.keys()))}')
    except KeyError as err:
        # ignore the queries that are not in the validation queries list
        pass

Processed queries: 1/477
Processed queries: 2/477
Processed queries: 3/477
Processed queries: 4/477
Processed queries: 5/477
Processed queries: 6/477
Processed queries: 7/477
Processed queries: 8/477
Processed queries: 9/477
Processed queries: 10/477
Processed queries: 11/477
Processed queries: 12/477
Processed queries: 13/477
Processed queries: 14/477
Processed queries: 15/477
Processed queries: 16/477
Processed queries: 17/477
Processed queries: 18/477
Processed queries: 19/477
Processed queries: 20/477
Processed queries: 21/477
Processed queries: 22/477
Processed queries: 23/477
Processed queries: 24/477
Processed queries: 25/477
Processed queries: 26/477
Processed queries: 27/477
Processed queries: 28/477
Processed queries: 29/477
Processed queries: 30/477
Processed queries: 31/477
Processed queries: 32/477
Processed queries: 33/477
Processed queries: 34/477
Processed queries: 35/477
Processed queries: 36/477
Processed queries: 37/477
Processed queries: 38/477
Processed queries: 39